<a href="https://colab.research.google.com/github/astidwis7/arabic_completion/blob/main/arabicCompletion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Python program to complete bald Arabic using Bidirectional LSTM***

Link to open the dataset --> [arabic](https://drive.google.com/drive/folders/1s9HRTWmM2t5DR1-lIyzed09Z5bwjysUN?usp=drive_link)

In [ ]:
# retrieve arabic data

import pandas as pd

ribasti = pd.read_csv('arabic.csv')

In [ ]:
# windowing data 

import numpy as np

korpus = []
for _ in ribasti['0']:
  korpus.append(str(_))
korpusan = " ".join(korpus)

korpus_harakat = []
for _ in ribasti['1']:
  korpus_harakat.append(str(_))
korpusan_harakat = " ".join(korpus_harakat)

korpus2 = []
korpusan2 = korpusan.split()
for i in range(len(korpusan2)-7):
  korpus2.append(" ".join(korpusan2[i:i+7]))

korpus2_h = []
korpusan2_h = korpusan_harakat.split()
for i in range(len(korpusan2_h)-7):
  korpus2_h.append(" ".join(korpusan2_h[i:i+7]))

In [ ]:
# tokenization with keras tokenizer

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

token_x = Tokenizer()
token_y = Tokenizer()

token_x.fit_on_texts(korpus2)
token_y.fit_on_texts(korpus2_h)

x_seq = token_x.texts_to_sequences(korpus2)
y_seq = token_y.texts_to_sequences(korpus2_h)

x_seq_padding = pad_sequences(x_seq, maxlen=7, padding="post")
x = np.asarray(x_seq_padding).astype(float)

y_seq_padding = pad_sequences(y_seq, maxlen=7, padding="post")
y =  np.asarray(y_seq_padding)

In [ ]:
# Bidirectional LSTM model initiation

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, TimeDistributed, Bidirectional

embedding_dim = 16

classifier = Sequential()

classifier.add(Embedding(int(np.max(x)+1), embedding_dim, input_length=x.shape[1], input_shape=x.shape[1:]))
classifier.add(Bidirectional(LSTM(128, return_sequences=True)))
classifier.add(TimeDistributed(Dense(64, activation='relu')))
classifier.add(TimeDistributed(Dense(int(np.max(y)+1), activation='softmax')))

classifier.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# models training

classifier.fit(x,y, batch_size=2, epochs=6, validation_split=0.1)

Epoch 1/6
57695/57695 [==============================] - 1368s 24ms/step - loss: 3.4395 - accuracy: 0.4448 - val_loss: 3.7557 - val_accuracy: 0.4795
Epoch 2/6
57695/57695 [==============================] - 1364s 24ms/step - loss: 2.2980 - accuracy: 0.6036 - val_loss: 3.3963 - val_accuracy: 0.5301
Epoch 3/6
57695/57695 [==============================] - 1373s 24ms/step - loss: 1.8760 - accuracy: 0.6563 - val_loss: 3.2746 - val_accuracy: 0.5388
Epoch 4/6
57695/57695 [==============================] - 1381s 24ms/step - loss: 1.6167 - accuracy: 0.6892 - val_loss: 3.2174 - val_accuracy: 0.5509
Epoch 5/6
57695/57695 [==============================] - 1396s 24ms/step - loss: 1.4430 - accuracy: 0.7138 - val_loss: 3.2016 - val_accuracy: 0.5573
Epoch 6/6
57695/57695 [==============================] - 1425s 25ms/step - loss: 1.3269 - accuracy: 0.7321 - val_loss: 3.2120 - val_accuracy: 0.5601


In [ ]:
# save Bidirectional LSTM model

# classifier.save_weights('arabic.h5')

In [ ]:
# predict using saved model

classifier.load_weights('arabic.h5')

kalimat = token_x.texts_to_sequences(['b sm {ll~h {l r~Hm`n {l r~Hym'])
kalimat_pad = pad_sequences(kalimat, maxlen=7, padding="post")

pred = classifier.predict(kalimat_pad)

pred_ = []
for i in range(7):
  pred_.append([np.argmax(pred[0][i])])

hasil = " ".join(token_y.sequences_to_texts(pred__))
hasil


1/1 [==============================] - 0s 26ms/step


'l r ahoma ni l r ahiymi'